In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import requests
import datetime
import time
import json

def get_train_routes_with_session(code_from, code_to, date, with_seats=True):
    """Получение маршрутов с использованием сессии для сохранения состояния."""
    base_url = "https://pass.rzd.ru/timetable/public/ru"
    session = requests.Session()  # Инициализируем сессию

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Referer": "https://pass.rzd.ru/",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive"
    }

    params = {
        "layer_id": 5827,
        "dir": 0,
        "tfl": 1,
        "checkSeats": 1 if with_seats else 0,
        "code0": code_from,
        "code1": code_to,
        "dt0": "24.12.2024"
    }


    response = session.get(base_url, params=params, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            rid = data.get("RID")
            if not rid:
                print("RID нет, поэтому второй запрос не получится сделать")
                print("Ответ первого запроса:", data)
                return None

            print(f"Первый запрос выполнен успешно, JSON: {data}")

            time.sleep(1)

            second_url = f"{base_url}?layer_id=5827&rid={rid}"
            second_response = session.get(second_url, headers=headers, params=params)
            if second_response.status_code == 200:
                data = second_response.json()
                print(f"Второй запрос выполнен успешно, JSON: {data}")

                try:
                    return data, second_response
                except ValueError:
                    print("Ошибка преобразования ответа второго запроса в JSON.")
                    print(second_response.text)
                    return None

            else:
                print(f"Что-то пошло не так при втором запросе, статус ошибки:: {second_response.status_code}, причина: {second_response.reason}")
                return None
        except ValueError:
            print("Ошибка преобразования ответа первого запроса в JSON.")
            print(response.text)
            return None
    else:
        print(f"Что-то пошло не так при первом запросе, статус ошибки: {response.status_code}\nПричина: {response.reason}")
        return None


def get_station_code(station_name_part):
    """
    Получает код станции по части её названия.
    :param station_name_part: Подстрока названия станции (не менее 2 символов).
    :return: Список найденных станций в формате [(название, код), ...].
    """
    if len(station_name_part) < 2:
        print("Название станции должно содержать не менее 2 символов.")
        return None

    base_url = "https://pass.rzd.ru/timetable/public/ru"
    session = requests.Session()  # Создаем сессию

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Referer": "https://pass.rzd.ru/",
        "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "Connection": "keep-alive",
        "Accept": "application/json, text/javascript, */*; q=0.01"
    }

    params = {  # Эндпоинт для поиска станций
        'stationNamePart' : station_name_part,
    }

    try:
        # Отправляем запрос через сессию
        response = session.get(base_url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            print("Ответ от сервера:", data)

            # Парсим список станций
            stations = data.get("list", [])
            if not stations:
                print("Станции не найдены.")
                return None

            # Возвращаем список станций в формате [(название, код)]
            return [(station["station"], station["code"]) for station in stations]
        else:
            print(f"Ошибка запроса: {response.status_code} {response.reason}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе: {e}")
        return None
    except ValueError:
        print(f"Ошибка при преобразовании ответа в JSON. {response.json()}")
        return None


with open("codes.json", 'r') as file:
    file = json.load(file)
    code_from = file.get("москва")
    code_to = file.get("казань")
    file1 = get_train_routes_with_session(code_from, code_to,'20.12.2024')
    print(file1)




Первый запрос выполнен успешно, JSON: {'result': 'RID', 'RID': 36810595764, 'timestamp': '07.12.2024 15:23:55.748'}
Второй запрос выполнен успешно, JSON: {'result': 'OK', 'tp': [{'from': 'МОСКВА', 'fromCode': 2000000, 'where': 'КАЗАНЬ', 'whereCode': 2060615, 'date': '24.12.2024', 'noSeats': False, 'defShowTime': 'local', 'state': 'Trains', 'list': [{'number': '150Х', 'number2': '150Х', 'type': 0, 'typeEx': 0, 'depth': 89, 'new': False, 'elReg': True, 'deferredPayment': False, 'varPrice': False, 'code0': 2000003, 'code1': 2060500, 'bEntire': True, 'trainName': '', 'brand': '', 'carrier': 'ФПК', 'route0': 'МОСКВА КАЗ', 'route1': 'КАЗАНЬ ПАС', 'routeCode0': 2000003, 'routeCode1': 2060500, 'trDate0': '24.12.2024', 'trTime0': '00:20', 'station0': 'МОСКВА КАЗАНСКАЯ (КАЗАНСКИЙ ВОКЗАЛ)', 'station1': 'КАЗАНЬ ПАСС', 'date0': '24.12.2024', 'time0': '00:20', 'date1': '24.12.2024', 'time1': '13:27', 'timeInWay': '13:07', 'flMsk': 3, 'train_id': 0, 'cars': [{'carDataType': 1, 'itype': 6, 'type': 'Лю

In [ ]:
response.status_code
response.url
response.json()

{'result': 'OK',
 'tp': [{'from': 'САНКТ-ПЕТЕРБУРГ ЛАДОЖ.',
   'fromCode': 2004006,
   'where': 'КИРОВ ПАСС',
   'whereCode': 2060600,
   'date': '24.12.2024',
   'noSeats': False,
   'defShowTime': 'local',
   'state': 'Trains',
   'list': [{'number': '131Г',
     'number2': '131Г',
     'type': 0,
     'typeEx': 0,
     'depth': 89,
     'new': False,
     'elReg': True,
     'deferredPayment': False,
     'varPrice': False,
     'code0': 2004006,
     'code1': 2060600,
     'bEntire': True,
     'trainName': '',
     'brand': '',
     'carrier': 'ФПК',
     'route0': 'С-ПЕТ-ЛАД',
     'route1': 'ИЖЕВСК',
     'routeCode0': 2004006,
     'routeCode1': 2060150,
     'trDate0': '24.12.2024',
     'trTime0': '13:10',
     'station0': 'САНКТ-ПЕТЕРБУРГ (ЛАДОЖСКИЙ ВОКЗАЛ)',
     'station1': 'КИРОВ ПАСС',
     'date0': '24.12.2024',
     'time0': '13:10',
     'date1': '25.12.2024',
     'time1': '10:10',
     'timeInWay': '21:00',
     'flMsk': 3,
     'train_id': 0,
     'cars': [{'carDat

In [124]:
print(response.status_code)
response.reason

200


'OK'

## тест код для обновления данных и уведомлений

In [5]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install winsound

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement winsound (from versions: none)
ERROR: No matching distribution found for winsound


In [ ]:
# coding=utf-8
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time, re
# import winsound


class Rzdtemp():
    def __init__(self, logger):
        self.logger = logger
    def setUp(self):
        self.driver = webdriver.Firefox()
        self.driver.implicitly_wait(30)
        self.base_url = "http://ticket.rzd.ru/"
        self.verificationErrors = []

    def test_rzdtemp(self):

        self.logger.info('Вход...')

        driver = self.driver
        driver.get(self.base_url + "/static/public/ticket?STRUCTURE_ID=2")
        driver.find_element_by_link_text("Вход").click()

        self.logger.info('Ввод логина и пароля...')

        driver.find_element_by_id("j_username").clear()
        driver.find_element_by_id("j_username").send_keys("username")
        driver.find_element_by_id("j_password").clear()
        driver.find_element_by_id("j_password").send_keys("password")
        driver.find_element_by_id("other").click()

        self.logger.info('Меню покупки билетов...')

        driver.find_element_by_link_text("Покупка билета").click()

        self.logger.info('Ввод данных...')

        driver.find_element_by_id("fromInput").clear()
        driver.find_element_by_id("fromInput").send_keys(u'КАЗАНЬ ПАСС')
        driver.find_element_by_id("whereInput").clear()
        driver.find_element_by_id("whereInput").send_keys(u'МОСКВА КАЗАНСКАЯ')
        driver.find_element_by_id("forwardDate").clear()
        driver.find_element_by_id("forwardDate").send_keys(u'02.09.2012')
        driver.find_element_by_id("ticket_button_submit").click()

        time.sleep(40)

        self.logger.info('Поиск нужных билетов...')
        rawhtml = driver.find_element_by_id('ajaxTrainTable').get_attribute("innerHTML")

        if u'Плацкартный' in rawhtml:
            self.logger.info('!!!ЕСТЬ ПЛАЦКАРТ!!!')
            strlist = [x.strip() for x in rawhtml.split('n') if x.strip()!=u'']
            #print strlist
            train = ''
            for i,x in enumerate(strlist):
                if x == u'<div class="wotnumarrow">':
                    train = strlist[i+1].replace('<span><b>','')
                if x == u'Плацкартный':
                    #включаем сигнал тревоги
                    # winsound.PlaySound('alarma.ogg', winsound.SND_NOWAIT)
                    self.logger.info(u'Поезд-%s Число-%s %s' % ( train, strlist[i+3].replace('<b>','').replace('</b>',''),
                    strlist[i+5].replace('<td><span>','').replace('</span></td>','')))

        elif u'Сидячий' in rawhtml:
            self.logger.info('Только сидячие...')
        elif u'Купе' in rawhtml:
            self.logger.info('Только купе...')

        self.logger.info('Выход...')

        driver.find_element_by_link_text("Выход").click()

    def tearDown(self):
        self.logger.info('Закрываем браузер...')
        self.driver.close()
        self.driver.quit()


ModuleNotFoundError: No module named 'selenium'

In [ ]:
import logging

logger = logging.getLogger(__name__)
temp = Rzdtemp(logger)
temp.setUp()

WebDriverException: Message: Process unexpectedly closed with status 1


In [ ]:
temp.test_rzdtemp()

AttributeError: 'Rzdtemp' object has no attribute 'driver'